<a href="https://colab.research.google.com/github/wubinstu/roop/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pre roop

!git clone https://github.com/wubinstu/roop.git
workdir = "/content/roop"
%pip install -r {workdir}/requirements.txt


source = f"{workdir}/media/*.jpg"
target = f"{workdir}/media/in.mp4"
output = f"{workdir}/media/out.mp4"

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# cp file
!cp -rf /content/drive/MyDrive/? {target}

In [ ]:
# starting

Device = "cuda" #@param ["cuda", "cpu"]
Processor = "face_swapper" #@param ["face_swapper face_enhancer", "face_swapper","face_enhancer"]
VideoEncoder = "libx264" #@param ["libx264", "libx265","ibvpx-vp9"]
VideoQuality = "10" #@param {type:"string"}

KeepFPS = False #@param {type:"boolean"}
KeepAudio = True #@param {type:"boolean"}
KeepFrames = False #@param {type:"boolean"}
ManyFaces = False #@param {type:"boolean"}

KeepFPS ="--keep-fps" if KeepFPS==True else ""
KeepAudio ="--keep-audio" if KeepAudio==True else ""
KeepFrames ="--keep-frames" if KeepFrames==True else ""
ManyFaces ="--many-faces" if ManyFaces==True else ""

cmd = f"{workdir}/run.py --execution-provider {Device} -s {source} -t {target} -o {output} --frame-processor {Processor} --video-encoder {VideoEncoder} --video-quality {VideoQuality} {KeepFPS} {KeepAudio} {KeepFrames} {ManyFaces}"
print("cmd:"+cmd)
!python $cmd
